In [1]:
# 데이터 경로 설정 및 데이터 불러오기
import os
os.chdir('./data')

import pandas as pd

#### 데이터 크기에 따른 모델 선택: *특징 개수가 매우 적은 경우*

In [2]:
# 데이터 불러오기
import numpy as np
df = pd.read_csv("Combined_Cycle_Power_Plant.csv")
df.shape

(9568, 5)

In [3]:
# 특징과 라벨 분리
X = df.drop('EP', axis = 1)
Y = df['EP']

In [5]:
# 모델 리스트 정의
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor as MLP
from sklearn.linear_model import LinearRegression as LR
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.neighbors import KNeighborsRegressor as KNR

# 공정한 비교를 위해 전부 default 값을 사용
# random_state가 있는 모델은 모두 같은 값으로 설정

SVR_model = SVR()
MLP_model = MLP(random_state = 100)
LR_model = LR()
DTR_model = DTR(random_state = 100)
RFR_model = RFR(random_state = 100)
KNR_model = KNR()

model_list = [SVR_model, MLP_model, LR_model, DTR_model, RFR_model, KNR_model]
model_name_list = ['SVR', 'MLP', 'LR', 'DTR', 'RFR', 'KNR']

In [6]:
# 모델별 k겹 교차 검증 기반(k = 5)의 MAE값 계산
from sklearn.model_selection import cross_val_score
for (model, model_name) in zip(model_list, model_name_list):
    score = -cross_val_score(model, X, Y, cv = 5, scoring = 'neg_mean_absolute_error').mean() # -MAE이므로 다시 -를 붙인 것
    print(model_name, score)

SVR 11.15624063928403
MLP 3.942187462994771
LR 3.6282513807290444
DTR 3.0813730587778396
RFR 2.3680572129208857
KNR 2.9176435395285303


샘플 수는 많고, 특징이 적은 경우 => 적당히 복잡한 모델 RFR, KNR이 적절
 - 특징이 많다 => 복잡한 모델은 의미X
     - 특히 SVR은 데이터가 어지간히 복잡하지 않으면 좋은 성능 X
 - 샘플 수가 많기때문에 RFR에서 각 샘플들을 분할(배깅-앙상블 모델)하더라도 모델 학습이 잘 됐을 것이라 추정됨
     - xgbm, lgbm을 써도 성능이 좋았을 테지만 rfr보단 좋았을 것 같지 않음

#### *샘플이 매우 적고, 특징이 상대적으로 많은 경우*

In [7]:
# 데이터 불러오기
import pandas as pd
df = pd.read_csv("baseball.csv")

In [8]:
df.shape

(337, 17)

In [9]:
# 특징과 라벨 분리
X = df.drop('Salary', axis = 1)
Y = df['Salary']

In [10]:
# 모델 리스트 정의
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor as MLP
from sklearn.linear_model import LinearRegression as LR
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.neighbors import KNeighborsRegressor as KNR

# 공정한 비교를 위해 전부 default 값을 사용
# random_state가 있는 모델은 모두 같은 값으로 설정

SVR_model = SVR()
MLP_model = MLP(random_state = 100)
LR_model = LR()
DTR_model = DTR(random_state = 100)
RFR_model = RFR(random_state = 100)
KNR_model = KNR()

model_list = [SVR_model, MLP_model, LR_model, DTR_model, RFR_model, KNR_model]
model_name_list = ['SVR', 'MLP', 'LR', 'DTR', 'RFR', 'KNR']

In [11]:
# 모델별 k겹 교차 검증 기반(k = 5)의 MAE값 계산
# SVR의 성능이 최악
# MLP에서 max_iter 이슈 발생
from sklearn.model_selection import cross_val_score
for (model, model_name) in zip(model_list, model_name_list):
    score = - cross_val_score(model, X, Y, cv = 5, scoring = 'neg_mean_absolute_error').mean() # -MAE이므로 다시 -를 붙인 것
    print(model_name, score)

SVR 940.1650397834104


C:\Users\hyun\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\hyun\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\hyun\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\hyun\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\hyun\lib\site-packa

MLP 710.1436141313578
LR 539.567114538396
DTR 573.3383230904303
RFR 467.4962985074626
KNR 651.0923792800702


샘플 수는 적고, 특징이 많은 경우 => 단순한 모델 LR, RFR이 적절
 - 여기서 RFR이 또 성능이 좋게 나온 이유는 배깅을 할 때 feature들까지 분할+랜덤되기때문에 샘플도 적고 특징도 적은 형태였기 때문에 좋지않았나